In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error


In [3]:
training_file = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test_file = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")


In [4]:
training_file.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [29]:
training_file.shape

(2834, 6)

In [5]:
data = training_file[['excerpt', 'target']]
data = data.sample(frac=1).reset_index(drop=True)
excerpt, targets = training_file['excerpt'].values, training_file['target'].values

t_X, v_X = excerpt[:2750], excerpt[2750:]
t_Y, v_Y = targets[:2750], targets[2750:]

print(t_X.shape, v_X.shape)
print(t_Y.shape, v_Y.shape)


(2750,) (84,)
(2750,) (84,)


In [16]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [19]:
models={"LR":LinearRegression(),
        "rdg":Ridge(),
        "rf":RandomForestRegressor(n_estimators=100),
        "Adb":AdaBoostRegressor(n_estimators=120)
        }

In [37]:
def train(model,vec):
    model_pipeline=make_pipeline(
    vec,
    model
    )
    model_pipeline.fit(t_X, t_Y)

    preds = model_pipeline.predict(v_X)
    mse_loss = mean_squared_error(v_Y, preds)

    print(f"MSE Loss using {name} model is: {mse_loss}")


<h2>Using Countvectorizer

In [39]:
for name,model in zip(models.keys(),models.values()):
        train(model,vec=CountVectorizer(binary=True))

MSE Loss using LR model is: 0.7889389346352619
MSE Loss using rdg model is: 0.7758521744843346
MSE Loss using rf model is: 0.7450382133129176
MSE Loss using Adb model is: 0.7851758461791363


<h2>Using TfidfVectorizer 

In [40]:
for name,model in zip(models.keys(),models.values()):
        train(model, vec=TfidfVectorizer(binary=True, ngram_range=(1, 1)))

MSE Loss using LR model is: 0.6308449498044326
MSE Loss using rdg model is: 0.5969889898401092
MSE Loss using rf model is: 0.6269905293155892
MSE Loss using Adb model is: 0.6433095298500602


In [45]:
#From above we can see that ridge regression has minimum loss
final_model=make_pipeline(
                TfidfVectorizer(binary=True, ngram_range=(1, 1)),
                models['rdg'])
final_model.fit(t_X, t_Y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(binary=True)),
                ('ridge', Ridge())])

In [46]:
test = test_file[['id', 'excerpt']]
test_ids = test['id'].tolist()
test_text = test['excerpt'].values

test_preds_linear = final_model.predict(test_text)

submission = pd.DataFrame()
submission['id'] = test_ids
submission['target']=test_preds_linear
submission.to_csv("submission.csv", index=None)